# Designing Gain and Offset in Thirty Seconds
Based on https://www.ti.com/lit/an/sloa097/sloa097.pdf

In [1]:
import numpy as np
import sympy as smp
import matplotlib.pyplot as plt
from dataclasses import dataclass

In [2]:
def calculate_m(Vin, Vout):
    m = (Vout.full_scale - Vout.zero_scale) / (Vin.full_scale - Vin.zero_scale)
    return m

def calculate_b(Vin, Vout):
    m = calculate_m(Vin, Vout)
    b = Vout.zero_scale - m*Vin.zero_scale
    return b

@dataclass
class VoltageSpan:
    zero_scale: float
    full_scale: float

In [23]:
def calculate_pos_m_pos_b(Vin, Vout, Rf, R1, Vref):
    m = calculate_m(Vin, Vout)
    b = calculate_b(Vin, Vout)
    assert m>=0
    assert b>=0
    R2 = Vref*R1*m/b
    Rg = R2*Rf / (m*(R1+R2) - R2)
    return {"R2": R2, "Rg": Rg}

def calculate_pos_m_neg_b(Vin, Vout, Rf, Vref):
    m = calculate_m(Vin, Vout)
    b = calculate_b(Vin, Vout)
    assert m >= 0
    assert b <= 0
    Rg = Rf / (m-1)
    Rg2 = Rg / 10
    Rg1 = Rg-Rg2
    Vref_ = abs(b)*Rg1 / (Rg1+Rf)
    R1 = Rg2*(Vref - Vref_) / Vref_
    return {'Rg': Rg, 'Rg1': Rg1, 'Rg2': Rg2, "R1": R1, "Vref'": Vref_}

In [26]:
Vin = VoltageSpan(0,2)
Vout = VoltageSpan(8,10)
m = calculate_m(Vin, Vout)
b = calculate_b(Vin, Vout)
print(f"m = {m:.1f}, b = {b:.1f}")

m = 1.0, b = 8.0


In [27]:
calculate_pos_m_pos_b(Vin, Vout, Rf = 2e3, Vref = 10, R1 = 1e3)

{'R2': 1250.0, 'Rg': 2500.0}